In [1]:
# importing function from a different ipny 
from ipynb.fs.full.dataset import spam_test_train_set
from ipynb.fs.full.dataset import confusion
import pandas as pd
import collections
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000

In [2]:
[train_email, test_email] = spam_test_train_set()

2018-12-05 21:00:18,853   ERROR:mailparser.utils:273:      parse_received More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from anderson.doramail.com 207.12.141.100 by coliform.funmail.co.uk InterMail vK.4.04.00.00 658-589-341 license 893694jz34ml9wal1qv276h1o63n3u76 with ESMTP id <20034613789721.SXCH195.coliform@anderson.doramail.com> for <kruscit@enron.com>; Sun, 5 Aug 2001 20:34:59 -0600 Received: from Matthew 206.81.96.2 by anderson.doramail.com iPlanet Messaging Server 5.2 HotFix 1.14 built Mar 18 2003 with ESMTPA id <0OMZ008USAR04O@anderson.doramail.com> for kruscit@enron.com; Sun, 5 Aug 2001 23:34:59 -0300 IDT
2018-12-05 21:00:19,145   ERROR:mailparser.utils:273:      parse_received More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from anderson.doramail.com 207

In [3]:
train_email.subject.iloc[:20]

4                        CP&L tariff changes (ER01-1807)
5                            IM draft - Florida Projects
15              News About Your VentureWire Subscription
20                                            8/3/01 MPR
47     TRRC Chairman MICHAEL WILLIAMS and Land Commis...
54         FW: Day of Caring - 5719 Ettrick, Aug 8, 2001
56                              RE: Practice Sunday at 1
58              News About Your VentureWire Subscription
65                                                   RE:
67                                       Capacity Center
69     TRRC Chairman MICHAEL WILLIAMS and Land Commis...
71     Order re. transfer of operational control to MISO
72                            Billing Procedure Question
75                        Editorial on Emissions Trading
77                                              Schedule
79                                Practice Thurs. at 6PM
87            FW: Aug. 6 -- BNA, Inc. Daily Labor Report
95     PJM Interconnection LLC 

#### RE: appears at locations 7 and 9 TRRC 5 and 11
#### Notice RE: and re. are different. 
#### Spacy tokenize RE into 'RE' and ':'

In [9]:
## New code 
def iter_spacy(strs, tokenizer):
    words = []
    for s in strs:
        doc = nlp(s)
        tokens = [str(token) for token in doc]
        counts = collections.Counter(tokens)
        yield counts

In [10]:
bows_spacy = pd.DataFrame(iter_spacy(train_email.subject.iloc[:20], tokenizer )).fillna(0).astype(int)

In [8]:
bows_spacy.head(15)

,,&,(,),",",-,--,.,/,1,1807,2001,2327,5719,6,6PM,8,8/3/01,:,About,Aug,Aug.,BNA,Billing,CP&L,Capacity,Caring,Cash,Center,Chairman,Commissioner,DAVID,DEWHUR,Daily,Data,Day,ER01,Earnings,Editorial,Emissions,Entity,Ettrick,FW,Florida,Flow,Functional,IM,Inc.,Incl,Interconnection,LLC,Labor,Land,MICHAEL,MISO,MPR,Models,NNG,News,Non,OATT,Order,PJM,Practice,Procedure,Projects,Question,Quote,RE,Report,ST,Schedule,Subscription,Sunday,TRRC,TW,Thurs,Trading,Update,VentureWire,WILLIAMS,Your,and,at,changes,control,day,draft,of,on,operational,re,revisions,tariff,the,to,transfer
0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,2,1,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
